<a href="https://colab.research.google.com/github/sergiocostaifes/PPCOMP_DM/blob/main/notebooks/03_windowing_episodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================
# 03_windowing_episodes.ipynb
# Série por hora + detecção de episódios (baseline μ+2σ)
# =========================

import pandas as pd
import numpy as np

CLEAN_PARQUET = PROCESSED_PATH / "google_trace_clean.parquet"
assert CLEAN_PARQUET.exists(), f"Não achei: {CLEAN_PARQUET}"

df = pd.read_parquet(CLEAN_PARQUET)
log(f"Dataset limpo carregado: {df.shape}")

# 1) Série f(h): falhas por hora, incluindo horas com zero falhas
hmin = int(df["hour"].min())
hmax = int(df["hour"].max())

fail_by_hour_full = (
    df[df["failed"] == 1]
    .groupby("hour")
    .size()
    .reindex(range(hmin, hmax + 1), fill_value=0)
)

series = fail_by_hour_full  # hour 0 já foi removida no notebook anterior

log(f"Intervalo horas: {hmin}..{hmax}")
log(f"Total horas na série: {len(series)}")
log(f"Total falhas: {int(series.sum())}")
log(f"Pico máximo: {int(series.max())}")

# 2) Threshold estatístico T = μ + 2σ
mu = float(series.mean())
sigma = float(series.std())
threshold = mu + 2*sigma

log(f"μ (média): {mu:.2f}")
log(f"σ (desvio padrão): {sigma:.2f}")
log(f"T = μ + 2σ: {threshold:.2f}")

# 3) Detectar episódios como intervalos contínuos acima do threshold
episodes = []
start = None

for h, val in series.items():
    if val > threshold:
        if start is None:
            start = int(h)
    else:
        if start is not None:
            episodes.append((start, int(h) - 1))
            start = None

if start is not None:
    episodes.append((start, int(series.index.max())))

log(f"Número de episódios detectados: {len(episodes)}")
log(f"Primeiros episódios: {episodes[:10]}")

# 4) Tabela de episódios (métricas)
rows = []
for (hs, he) in episodes:
    seg = series.loc[hs:he]
    rows.append({
        "episode_id": len(rows) + 1,
        "start_hour": hs,
        "end_hour": he,
        "duration_hours": int(he - hs + 1),
        "max_failures_in_hour": int(seg.max()),
        "mean_failures_in_episode": float(seg.mean()),
        "total_failures_in_episode": int(seg.sum()),
        "threshold": float(threshold),
        "mu": float(mu),
        "sigma": float(sigma),
    })

episodes_df = pd.DataFrame(rows)

# 5) Salvar artefatos
WINDOW_BASE = FEATURES_PATH / "failures_by_hour.parquet"
EPISODES_OUT = FEATURES_PATH / "episodes_detected.parquet"

series.reset_index().rename(columns={"index": "hour", 0: "failures"}).to_parquet(WINDOW_BASE, index=False, compression="snappy")
episodes_df.to_parquet(EPISODES_OUT, index=False, compression="snappy")

log(f"Salvo: {WINDOW_BASE}")
log(f"Salvo: {EPISODES_OUT}")

episodes_df.sort_values("max_failures_in_hour", ascending=False).head(10)